### Import Libraries

In [45]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [46]:
import os
os.getcwd()

'D:\\Ngoding\\python'

### Import Data from CSV to DataFrame

In [47]:
df_csv = pd.read_csv('Online Retail Data.csv', header=0)
df_csv

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


In [48]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


### Data Cleansing

In [49]:
df_clean = df_csv.copy()

# create the date column
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64[ns]')

# remove all rows without customer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]

# remove all rows without product_name
df_clean = df_clean[~df_clean['product_name'].isna()]

# make all product_name entries lowercase
df_clean['product_name'] = df_clean['product_name'].str.lower() 

# remove all rows with product_code or product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) | 
                    (~df_clean['product_name'].str.lower().str.contains('test'))]

# make the order_status column with value 'cancelled' if the order_id starts with the letter 'c'
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')

# convert negative quantity values to positive
df_clean['quantity'] = df_clean['quantity'].abs()

# remove rows with a negative price
df_clean = df_clean[df_clean['price']>0]

# create the amount value ( multiplication between quantity and price)
df_clean['amount'] = df_clean['quantity'] * df_clean['price']

# replace the product_name of product_code that has multiple product_names with the most frequently occurring product_name
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')

# convert customer_id to a string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)

# remove the outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,order_status,amount
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,2010-01-04,cancelled,4.25
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,91.80
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,51.00
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,5.10
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,8.50
...,...,...,...,...,...,...,...,...,...,...
358464,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,2010-12-23,delivered,1.25
358465,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
358466,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
358467,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,2010-12-23,delivered,9.90


### Create RFM Segmentation

#### Aggregate transaction data into a summary of total transactions (orders), total order value, and the last order date for each user

In [50]:
df_user = df_clean.groupby('customer_id', as_index=False).agg(order_count=('order_id', 'nunique'), 
                                                              max_order_date=('date', 'max'), total_order_value=('amount','sum'))
df_user

,customer_id,order_count,max_order_date,total_order_value
0,12346.0,5,2010-10-04,602.40
1,12608.0,1,2010-10-31,415.79
2,12745.0,2,2010-08-10,723.85
3,12746.0,2,2010-06-30,266.35
4,12747.0,19,2010-12-13,4094.79
...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77
3885,18284.0,2,2010-10-06,486.68
3886,18285.0,1,2010-02-17,427.00
3887,18286.0,2,2010-08-20,941.48


#### Create a column for the number of days since the last order

In [51]:
today = df_clean['date'].max()
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order
0,12346.0,5,2010-10-04,602.40,80
1,12608.0,1,2010-10-31,415.79,53
2,12745.0,2,2010-08-10,723.85,135
3,12746.0,2,2010-06-30,266.35,176
4,12747.0,19,2010-12-13,4094.79,10
...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31
3885,18284.0,2,2010-10-06,486.68,78
3886,18285.0,1,2010-02-17,427.00,309
3887,18286.0,2,2010-08-20,941.48,125


In [52]:
df_clean.describe()

,quantity,price,date,amount
count,358469.000000,358469.000000,358469,358469.000000
mean,9.362754,3.087634,2010-07-21 18:16:27.057737216,16.757486
min,1.000000,0.001000,2010-01-04 00:00:00,0.001000
25%,2.000000,1.250000,2010-04-25 00:00:00,4.200000
50%,4.000000,1.950000,2010-08-02 00:00:00,10.200000
75%,12.000000,3.750000,2010-10-25 00:00:00,17.700000
max,216.000000,311.990000,2010-12-23 00:00:00,313.200000
std,16.288556,5.502037,NaN,25.312974


#### Create bins for the number of days since the last order consisting of 5 bins, with boundaries at min, P20, P40, P60, P80, max, and label them from 1 to 5 from highest to lowest as recency scores

In [53]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score
0,12346.0,5,2010-10-04,602.40,80,2
1,12608.0,1,2010-10-31,415.79,53,3
2,12745.0,2,2010-08-10,723.85,135,2
3,12746.0,2,2010-06-30,266.35,176,1
4,12747.0,19,2010-12-13,4094.79,10,5
...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4
3885,18284.0,2,2010-10-06,486.68,78,2
3886,18285.0,1,2010-02-17,427.00,309,1
3887,18286.0,2,2010-08-20,941.48,125,2


In [54]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 167.2+ KB


#### Create bins for total transactions (orders) consisting of 5 bins, with boundaries at min, P20, P40, P60, P80, max, and label them from 1 to 5 from lowest to highest as frequency scores

In [55]:
df_user['frequency_score'] = pd.cut(df_user['order_count'],
                                    bins=[0,
                                          np.percentile(df_user['order_count'], 20),
                                          np.percentile(df_user['order_count'], 40),
                                          np.percentile(df_user['order_count'], 60),
                                          np.percentile(df_user['order_count'], 80),
                                          df_user['order_count'].max()],
                                    labels=[1, 2, 3, 4, 5],
                                    include_lowest=True).astype(int)
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score
0,12346.0,5,2010-10-04,602.40,80,2,4
1,12608.0,1,2010-10-31,415.79,53,3,1
2,12745.0,2,2010-08-10,723.85,135,2,2
3,12746.0,2,2010-06-30,266.35,176,1,2
4,12747.0,19,2010-12-13,4094.79,10,5,5
...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4,4
3885,18284.0,2,2010-10-06,486.68,78,2,2
3886,18285.0,1,2010-02-17,427.00,309,1,1
3887,18286.0,2,2010-08-20,941.48,125,2,2


In [56]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int32         
 6   frequency_score       3889 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), object(1)
memory usage: 182.4+ KB


#### Create bins for total order value consisting of 5 bins, with boundaries at min, P20, P40, P60, P80, max, and label them from 1 to 5 from lowest to highest as monetary scores.

In [57]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                   bins=[df_user['total_order_value'].min(),
                                         np.percentile(df_user['total_order_value'], 20),
                                         np.percentile(df_user['total_order_value'], 40),
                                         np.percentile(df_user['total_order_value'], 60),
                                         np.percentile(df_user['total_order_value'], 80),
                                         df_user['total_order_value'].max()],
                                   labels=[1, 2, 3, 4, 5],
                                   include_lowest=True).astype(int)
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score
0,12346.0,5,2010-10-04,602.40,80,2,4,3
1,12608.0,1,2010-10-31,415.79,53,3,1,2
2,12745.0,2,2010-08-10,723.85,135,2,2,3
3,12746.0,2,2010-06-30,266.35,176,1,2,2
4,12747.0,19,2010-12-13,4094.79,10,5,5,5
...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4,4,3
3885,18284.0,2,2010-10-06,486.68,78,2,2,3
3886,18285.0,1,2010-02-17,427.00,309,1,1,2
3887,18286.0,2,2010-08-20,941.48,125,2,2,4


In [58]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int32         
 6   frequency_score       3889 non-null   int32         
 7   monetary_score        3889 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(1)
memory usage: 197.6+ KB


#### Create a segment name column based on recency and frequency scores.

In [59]:
df_user['segment'] = np.select(
    [(df_user['recency_score']==5) & (df_user['frequency_score']>=4),
     (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score']>=4),
     (df_user['recency_score']>=4) & (df_user['frequency_score'].between(2, 3)),
     (df_user['recency_score']<=2) & (df_user['frequency_score']==5),
     (df_user['recency_score']==3) & (df_user['frequency_score']==3),
     (df_user['recency_score']==5) & (df_user['frequency_score']==1),
     (df_user['recency_score']==4) & (df_user['frequency_score']==1),
     (df_user['recency_score']<=2) & (df_user['frequency_score'].between(3, 4)),
     (df_user['recency_score']==3) & (df_user['frequency_score']<=2),
     (df_user['recency_score']<=2) & (df_user['frequency_score']<=2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists', "04-Can't Lose Them", '05-Need Attention',
     '06-New Customers', '07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating']
)
df_user



,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score,segment
0,12346.0,5,2010-10-04,602.40,80,2,4,3,08-At Risk
1,12608.0,1,2010-10-31,415.79,53,3,1,2,09-About to Sleep
2,12745.0,2,2010-08-10,723.85,135,2,2,3,10-Hibernating
3,12746.0,2,2010-06-30,266.35,176,1,2,2,10-Hibernating
4,12747.0,19,2010-12-13,4094.79,10,5,5,5,01-Champion
...,...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4,4,3,02-Loyal Customers
3885,18284.0,2,2010-10-06,486.68,78,2,2,3,10-Hibernating
3886,18285.0,1,2010-02-17,427.00,309,1,1,2,10-Hibernating
3887,18286.0,2,2010-08-20,941.48,125,2,2,4,10-Hibernating


In [60]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int32         
 6   frequency_score       3889 non-null   int32         
 7   monetary_score        3889 non-null   int32         
 8   segment               3889 non-null   object        
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(2)
memory usage: 228.0+ KB


#### Display a summary of the RFM segmentation (point 8) showing the number of users, the average and median of total orders, total order value, and the number of days since the last order

In [61]:
summary = pd.pivot_table(
    df_user,
    index='segment',
    values=['customer_id', 'day_since_last_order', 'order_count', 'total_order_value'],
    aggfunc={
        'customer_id': pd.Series.nunique,
        'day_since_last_order': ['mean', 'median'],
        'order_count': ['mean', 'median'],
        'total_order_value': ['mean', 'median']
    }
)

# Calculate the percentage of unique customers
summary['pct_unique'] = (summary['customer_id'] / summary['customer_id'].sum() * 100).round(1)

summary

customer_id day_since_last_order        order_count  \
                           nunique                 mean median        mean   
segment                                                                      
01-Champion                    550            10.618182    9.5   15.467273   
02-Loyal Customers             546            40.864469   37.0    8.767399   
03-Potential Loyalists         523            23.573614   24.0    2.829828   
04-Can't Lose Them              64           121.984375  112.5   11.375000   
05-Need Attention              176            58.613636   59.0    3.397727   
06-New Customers                50            14.220000   16.0    1.000000   
07-Promising                   142            32.760563   34.0    1.000000   
08-At Risk                     426           140.455399  120.0    4.136150   
09-About to Sleep              352            58.735795   58.0    1.417614   
10-Hibernating                1060           196.837736  199.0    1.313208   

                              total_order_value           pct_unique  
                       median              mean    median             
segment                                                               
01-Champion              10.0       5003.674245  2775.525       14.1  
02-Loyal Customers        7.0       2622.817826  1946.850       14.0  
03-Potential Loyalists    3.0        766.769828   622.070       13.4  
04-Can't Lose Them        9.5       2839.948125  2268.405        1.6  
05-Need Attention         3.0        989.232676   826.370        4.5  
06-New Customers          1.0        244.689000   193.675        1.3  
07-Promising              1.0        287.800282   238.440        3.7  
08-At Risk                4.0       1153.825683   875.430       11.0  
09-About to Sleep         1.0        448.229688   334.755        9.1  
10-Hibernating            1.0        343.083842   257.005       27.3

In [69]:
summary.style.set_table_styles(
    [
        {'selector': 'thead th',
         'props': [('background-color', 'lightblue'),
                   ('color', 'black'),
                   ('font-weight', 'bold'),
                   ('border', '2px solid black')]},

        {'selector': '.index_name',
         'props': [('background-color', 'lightblue'),
                   ('color', 'black'),
                   ('font-weight', 'bold'),
                   ('border', '2px solid black')]},

        {'selector': '.row_heading',
         'props': [('background-color', 'lightyellow'),
                   ('color', 'black'),
                   ('font-weight', 'bold'),
                   ('border', '2px solid black')]}
    ]
).set_properties(**{
    'background-color': 'white',
    'color': 'black',
    'border-color': 'lightblue',
    'border': '1px solid black',
    'font-weight': 'bold'
})
